In [56]:
!pip install praw wikipedia-api

In [2]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 30.1 MB/s eta 0:00:0000:0100:01
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 29.8 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)


In [10]:
!pip install tqdm

  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)


In [1]:
import praw
import pandas as pd
import datetime as dt
from tqdm import tqdm
import numpy as np
import time

In [2]:
import logging

handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
for logger_name in ("praw", "prawcore"):
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(handler)

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

/Users/ehsanulkabir/miniconda/envs/final582/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import wikipediaapi

# Initialize the Wikipedia API
wiki = wikipediaapi.Wikipedia('en')

In [5]:
import gzip

# Load the Wikipedia article titles into a set for efficient lookup
wiki_titles = set()
with gzip.open("enwiki-latest-all-titles-in-ns0.gz", "rt", encoding="utf-8") as f:
    for title in f:
        title = title.strip()
        title = title.replace("_", " ")
        wiki_titles.add(title.strip())

In [6]:
reddit = praw.Reddit(client_id = "JgHZRyaH_Ya0sSPJ4WB4ZA", client_secret="yNybsXSQCZVR-r-2mo0Ribuae-zKgw", user_agent="scrape")

In [7]:
subreddit_name = "IAmA"

In [8]:
subreddit = reddit.subreddit(subreddit_name)

In [10]:
# Get all posts from the subreddit
top_subreddit = subreddit.new(limit=None)


# get 1000 posts from subreddit
# also get the comments from each post
# get 10 comments from each post that are not longer than 1000 characters
posts = []
# comments = []
for post in tqdm(top_subreddit):
    post_container = {}
    post_container["title"] = post.title
    post_container["body"] = post.selftext
    post_container["upvotes"] = post.score
    post_container["downvotes"] = post.downs
    post_container["upvote_ratio"] = post.upvote_ratio
    post_container["url"] = post.url
    post_container["num_comments"] = post.num_comments
    post_container["created"] = post.created
    post_container["author"] = post.author
    post_container["author_flair_text"] = post.author_flair_text
    post_container["id"] = post.id
    ner_results = nlp(post.title)
    person_entities = [entity for entity in ner_results if entity['entity'] == 'I-PER' or entity['entity'] == 'B-PER']
    person_entities = [entity['word'] for entity in person_entities]
    poster_name = " ".join(person_entities)
    post_container["person_entities"] = poster_name

    post_container["has_wiki"] = poster_name in wiki_titles and len(poster_name) > 0 and len(poster_name.split()) > 1

    posts.append(post_container)
    # posts.append(post_container)

0it [00:00, ?it/s]Fetching: GET https://oauth.reddit.com/r/IAmA/new
Data: None
Params: {'limit': 1024, 'raw_json': 1}
Response: 200 (122866 bytes)
100it [00:06, 20.49it/s]Fetching: GET https://oauth.reddit.com/r/IAmA/new
Data: None
Params: {'limit': 1024, 'after': 't3_11t5otw', 'raw_json': 1}
Response: 200 (123557 bytes)
199it [00:12, 18.96it/s]Fetching: GET https://oauth.reddit.com/r/IAmA/new
Data: None
Params: {'limit': 1024, 'after': 't3_10b1o85', 'raw_json': 1}
Response: 200 (149099 bytes)
300it [00:19, 20.65it/s]Fetching: GET https://oauth.reddit.com/r/IAmA/new
Data: None
Params: {'limit': 1024, 'after': 't3_yv832k', 'raw_json': 1}
Response: 200 (150819 bytes)
400it [00:25, 21.23it/s]Fetching: GET https://oauth.reddit.com/r/IAmA/new
Data: None
Params: {'limit': 1024, 'after': 't3_xqdtf2', 'raw_json': 1}
Response: 200 (157436 bytes)
500it [00:32, 18.87it/s]Fetching: GET https://oauth.reddit.com/r/IAmA/new
Data: None
Params: {'limit': 1024, 'after': 't3_wbcrx5', 'raw_json': 1}
Respo

In [ ]:
posts

In [ ]:
for i in tqdm(range(len(posts))):
    post_container = posts[i]
    ner_results = nlp(post_container["title"])
    person_entities = [entity for entity in ner_results if entity['entity'] == 'I-PER' or entity['entity'] == 'B-PER']
    person_entities = [entity['word'] for entity in person_entities]
    poster_name = " ".join(person_entities)
    post_container["person_entities"] = poster_name

    post_container["has_wiki"] = poster_name in wiki_titles and len(poster_name) > 0 and len(poster_name.split()) > 1

    if post_container["has_wiki"]:
        post_container["wiki_url"] = "https://en.wikipedia.org/wiki/" + poster_name.replace(" ", "_")

        wiki_page = wiki.page(poster_name)
        post_container["wiki_summary"] = wiki_page.summary

        
        post_container["comments"] = []
        post = reddit.submission(id=post_container["id"])
        post.comments.replace_more(limit=100)
        for comment in post.comments.list():
            post_container["comments"].append(comment.body)

In [116]:
len(posts)

282

In [105]:
# Get all posts from the subreddit
top_subreddit = subreddit.top(limit=10)


# get 1000 posts from subreddit
# also get the comments from each post
# get 10 comments from each post that are not longer than 1000 characters
posts = []
# comments = []
for post in tqdm(top_subreddit):
    post_container = {}
    post_container["title"] = post.title
    post_container["body"] = post.selftext
    post_container["upvotes"] = post.score
    post_container["downvotes"] = post.downs
    post_container["upvote_ratio"] = post.upvote_ratio
    post_container["url"] = post.url
    post_container["num_comments"] = post.num_comments
    post_container["created"] = post.created
    post_container["author"] = post.author
    post_container["author_flair_text"] = post.author_flair_text
    post_container["id"] = post.id

    ner_results = nlp(post.title)
    person_entities = [entity for entity in ner_results if entity['entity'] == 'I-PER' or entity['entity'] == 'B-PER']
    person_entities = [entity['word'] for entity in person_entities]
    poster_name = " ".join(person_entities)
    post_container["person_entities"] = poster_name

    post_container["has_wiki"] = poster_name in wiki_titles and len(poster_name) > 0 and len(poster_name.split()) > 1

    if post_container["has_wiki"]:
        post_container["wiki_url"] = "https://en.wikipedia.org/wiki/" + poster_name.replace(" ", "_")

        wiki_page = wiki.page(poster_name)
        post_container["wiki_summary"] = wiki_page.summary

        
        post_container["comments"] = []
        post.comments.replace_more(limit=10)
        for comment in post.comments.list():
            post_container["comments"].append(comment.body)

    posts.append(post_container)

998it [47:02,  2.83s/it]


In [119]:
posts_df = pd.DataFrame(posts)

In [120]:
posts_df

,title,body,upvotes,downvotes,upvote_ratio,url,num_comments,created,author,author_flair_text,id,person_entities,has_wiki,wiki_url,wiki_summary,comments
0,2 yaers ago I was working after hours on my so...,IGN trailer: [https://www.youtube.com/watch?v=...,85,0,0.76,https://www.reddit.com/r/IAmA/comments/134hns3...,38,1.682933e+09,mikimiki1618,None,134hns3,,False,NaN,NaN,NaN
1,We’re experts in immunology at The University ...,Happy International Day of [Immunology](https:...,1422,0,0.87,https://www.reddit.com/r/IAmA/comments/132tfat...,513,1.682776e+09,UniOfManchester,,132tfat,,False,NaN,NaN,NaN
2,"I've been a Private Jet Pilot for 15 years, As...",I have been a professional private jet pilot f...,3888,0,0.90,https://www.reddit.com/r/IAmA/comments/132az7h...,1537,1.682719e+09,Legacy500Driver,:verified: Verified,132az7h,,False,NaN,NaN,NaN
3,"I’m Callie, Insider’s senior music reporter an...",I founded Insider’s music team in 2020. I most...,496,0,0.75,https://www.reddit.com/r/IAmA/comments/131xn11...,79,1.682696e+09,thisisinsider,:verified: Scheduled AMA,131xn11,Callie,False,NaN,NaN,NaN
4,IAmA the COO of Joyce Moving and Storage looki...,"**Hi Reddit, I'm Will Joyce Jr, the COO of Jo...",23,0,0.67,https://www.reddit.com/r/IAmA/comments/13242sx...,18,1.682703e+09,Joyce_Van_Lines,None,13242sx,,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,"Louis Rossman and iFixit here, making it legal...",**Who we are:**\n\n* kwiens: [Kyle Wiens](http...,19781,0,0.94,https://www.reddit.com/r/IAmA/comments/yytfe2/...,1025,1.668802e+09,kwiens,,yytfe2,Louis Ross ##man,False,NaN,NaN,NaN
278,[crosspost/ r/colorado] I’m Daniel Bishop. I’m...,[Click here to view the AMA and join in the co...,420,0,0.86,https://www.reddit.com/r/IAmA/comments/yyp24l/...,1,1.668791e+09,cryfi,None,yyp24l,Daniel Bishop Bishop Jim Bishop,False,NaN,NaN,NaN
279,"I'm Dr Joseph Davis, Medical Director at Cayma...",Hi! I'm Dr Joseph Davis. I am one of the world...,36,0,0.84,https://www.reddit.com/r/IAmA/comments/yyt49d/...,63,1.668802e+09,DrJosephDavisDO,,yyt49d,Joseph Davis,True,https://en.wikipedia.org/wiki/Joseph_Davis,Joe Davis (1901–1978) was a British snooker an...,"[For more AMAs on this topic, subscribe to r/I..."
280,"I'm John Swierk, assistant professor of Chemis...","Hello all, I've been studying the photochemis...",2115,0,0.90,https://www.reddit.com/r/IAmA/comments/yxw6oj/...,117,1.668706e+09,intengineering,None,yxw6oj,John S ##wi ##er ##k,False,NaN,NaN,NaN


In [108]:
posts_df.to_csv("all_1000_IAmA_posts.csv")

In [109]:
posts_df = posts_df[posts_df["has_wiki"] == True]

In [110]:
posts_df.to_csv("all_1000_IAmA_posts_with_wiki.csv")

In [101]:
posts_df.reset_index(inplace=True)

In [104]:
posts_df.iloc[113]

index                                                              975
title                Iam Independent Journalist Tim Pool, founding ...
body                 Howdy all,\n\nMy name is Tim Pool. In the past...
upvotes                                                          11969
downvotes                                                            0
upvote_ratio                                                      0.71
url                  https://www.reddit.com/r/IAmA/comments/5q3ck5/...
num_comments                                                      2037
created                                                   1485353324.0
author                                                         Timcast
author_flair_text                                                 None
id                                                              5q3ck5
person_entities                                               Tim Pool
has_wiki                                                          True
wiki_u

In [92]:
set(posts_df["person_entities"].to_numpy().tolist())

{'AJ Green',
 'Adam Savage',
 'Andrew Yang',
 'Andy Newman',
 'Andy Weir',
 'Barack Obama',
 'Bernie Sanders',
 'Bill Gates',
 'Bill Murray',
 'Brendan Fraser',
 'Bruce Campbell',
 'Charles Williams',
 'Charlie Duke',
 'Chris Hansen',
 'Chris Hunt',
 'Chris Pratt',
 'DJ Shadow',
 'Daniel Radcliffe',
 'Daniel Stern',
 'Danny Thompson',
 'Daryl Davis',
 'David Cross',
 'David Simon',
 'Don Lewis',
 'Donald Trump',
 'Drew Hansen',
 'Dylan Moran',
 'Edward Snow',
 'Elizabeth Smart',
 'Frank Oz',
 'Gordon Ramsay',
 'Guillermo del Toro',
 'H Jon Benjamin',
 'Ian Anderson',
 'James May',
 'Jason Rogers',
 'Jeff Bridges',
 'Jill Stein',
 'John Green',
 'John Oliver',
 'John Plant',
 'Jon Snow',
 'Jon Stewart',
 'Jordan B Peterson',
 'Jordan Prentice',
 'Katy Perry',
 'Ken Bone',
 'Ken Jennings',
 'Kim Jong',
 'Kristen Bell',
 'Leroy Jenkins',
 'Liam Gallagher',
 'Lisa Fletcher',
 'Marilyn Manson',
 'Martin Starr',
 'Matt Damon',
 'Max Brooks',
 'Michael Phelps',
 'Michael Stevens',
 'Mike Rowe

In [26]:
# get post from post id
post = reddit.submission(id="l9y3ud")

In [27]:
post.selftext

''

In [36]:
# Create a dataframe from the list of posts
posts_df = pd.DataFrame(posts)

In [47]:
# filter comments that are shorter than 100 characters or longer than 1000 characters
# also filter comments that do not contain the word "?"
posts_df["comments"] = posts_df["comments"].apply(lambda x: [comment for comment in x if len(comment) > 50 and len(comment) < 150 and "?" in comment])

In [29]:
posts_df.to_json("top_10_ama_posts.json")

In [64]:
posts_df.iloc[5]["title"]

'I’m Bill Gates, co-chair of the Bill & Melinda Gates Foundation. Ask Me Anything.'

In [57]:
import wikipediaapi

In [58]:
wiki_wiki = wikipediaapi.Wikipedia('en')

In [59]:
page = wiki_wiki.page('List_of_Academy_Award-winning_films')

In [63]:
page.text

'This is a list of Academy Award–winning films.\nIf a film won the Academy Award for Best Picture, its entry is listed in a shaded background with a boldface title.\nCompetitive Oscars are separated from non-competitive Oscars (i.e. Honorary Award, Special Achievement Award, Juvenile Award); as such, any films that were awarded a non-competitive award will be shown in brackets next to the number of competitive wins.\n\nStatistics\nAs of March 1, 2016\n\nTotal number of films: 1,239\nTotal number of Best Picture winners: 90\nTotal number of awards ceremonies: 90\nTotal number of awards associated with a film: 1,948 Competitive + 46 Honorary\nTotal number of nominations associated with a film: 4,403\nTotal number of Oscar Statuettes awarded: 2,810 Competitive + 49 Honorary = 2,859 in total\n\nSuperlatives\nFilms with the most awards: Ben-Hur (1959), Titanic (1997), and The Lord of the Rings: The Return of the King (2003) each earned 11 Academy Awards.\nFilms with the most nominations: Al